In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
# 形態素解析器 (Janomeを想定。実際はMeCabなどを使ってもOK)
from janome.tokenizer import Tokenizer

# 形態素解析の初期化
tokenizer = Tokenizer()

# ===============================================
# 共通機能 1: カテゴリのワンホットエンコーディング
# ===============================================

# OneHotEncoderのインスタンス（学習済みモデルを保存しておくために必要）
OHE_MODEL = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

def encode_category(df, column_name='ジャンル', fit_mode=False):
    """カテゴリ列をワンホットエンコーディングする。"""
    categories = df[[column_name]]
    
    if fit_mode:
        # 学習モード（特徴量行列作成時）
        return OHE_MODEL.fit_transform(categories)
    else:
        # 変換モード（ユーザーベクトル作成時）
        return OHE_MODEL.transform(categories)

# ===============================================
# 共通機能 2: 星の数の正規化 (Min-Maxスケーリング)
# ===============================================

# MinMaxScalerのインスタンス
SCALER_MODEL = MinMaxScaler(feature_range=(0, 1))

def scale_rating(df, column_name='平均評価', fit_mode=False):
    """星の数 (1.0~5.0) を0~1に正規化する。"""
    ratings = df[[column_name]]
    
    if fit_mode:
        # 学習モード（特徴量行列作成時）
        return SCALER_MODEL.fit_transform(ratings)
    else:
        # 変換モード（ユーザーベクトル作成時）
        return SCALER_MODEL.transform(ratings)

# ===============================================
# 共通機能 3: レビューのTF-IDFベクトル化の準備
# ===============================================

# TfidfVectorizerのインスタンス（学習済みモデルを保存しておくために必要）
# 実際にはここで形態素解析の前処理を組み込む必要があります
TFIDF_MODEL = TfidfVectorizer(max_features=5000) # 特徴量を5000単語に制限

def preprocess_review(text):
    """レビューテキストを形態素解析し、スペース区切り文字列に変換する前処理（ダミー）"""
    if pd.isna(text): return ""
    # 実際にはここで名詞、形容詞、動詞などを抽出する処理が入る
    tokens = [token.surface for token in tokenizer.tokenize(text) if token.part_of_speech.split(',')[0] in ['名詞', '形容詞']]
    return " ".join(tokens)

def vectorize_review(df, column_name='レビュー本文', fit_mode=False):
    """レビュー列をTF-IDFベクトルに変換する。"""
    # 前処理を実行
    processed_reviews = df[column_name].apply(preprocess_review)
    
    if fit_mode:
        # 学習モード（特徴量行列作成時）
        return TFIDF_MODEL.fit_transform(processed_reviews).toarray()
    else:
        # 変換モード（ユーザーベクトル作成時）
        return TFIDF_MODEL.transform(processed_reviews).toarray()

## pattern_total3


In [ ]:
def create_pattern1_features(df): #パターン1
    """
    パターン1: カテゴリ + 星の数 の特徴量行列を生成する
    """
    # 1. カテゴリの特徴量を生成 (fit_mode=Trueでモデルを学習)
    category_features = encode_category(df, column_name='ジャンル', fit_mode=True)

    # 2. 星の数の特徴量を生成 (fit_mode=Trueでモデルを学習)
    rating_features = scale_rating(df, column_name='平均評価', fit_mode=True)

    # 3. 結合（NumPyのconcatenateを使用）
    final_features = np.concatenate([category_features, rating_features], axis=1)
    
    return final_features

def create_pattern2_features(df): #パターン２
    """
    パターン2: カテゴリ + レビュー の特徴量行列を生成する
    """
    # 1. カテゴリの特徴量を生成 (fit_mode=Trueでモデルを学習)
    category_features = encode_category(df, column_name='ジャンル', fit_mode=True)

    # 2. レビューの特徴量を生成 (fit_mode=Trueでモデルを学習)
    review_features = vectorize_review(df, column_name='レビュー本文', fit_mode=True)

    # 3. 結合（NumPyのconcatenateを使用）
    final_features = np.concatenate([category_features, review_features], axis=1)
    
    return final_features

def create_pattern3_features(df): #パターン３
    """
    パターン3: カテゴリ + レビュー + 星の数 の特徴量行列を生成する
    """
    # 1. カテゴリの特徴量を生成 (fit_mode=Trueでモデルを学習)
    category_features = encode_category(df, column_name='ジャンル', fit_mode=True)

    # 2. 星の数の特徴量を生成 (fit_mode=Trueでモデルを学習)
    rating_features = scale_rating(df, column_name='平均評価', fit_mode=True)
    
    # 3. レビューの特徴量を生成 (fit_mode=Trueでモデルを学習)
    review_features = vectorize_review(df, column_name='レビュー本文', fit_mode=True)

    # 4. 結合（NumPyのconcatenateを使用）
    final_features = np.concatenate([category_features, rating_features, review_features], axis=1)
    
    return final_features



## ユーザーベクトル

In [ ]:
# ユーザーの入力例: category_query="和食", rating_query=4.0

def create_user_vector_pattern1(category_query, rating_query):
    """
    パターン1: カテゴリ + 星の数 のユーザーベクトルを生成する
    """
    # 1. カテゴリの処理（ワンホットエンコーディング）
    # 入力をPandas DataFrame形式に変換
    category_df = pd.DataFrame([category_query], columns=['ジャンル'])
    # 学習済みOHEモデルで変換
    user_category_vector = encode_category(category_df, column_name='ジャンル', fit_mode=False)

    # 2. 星の数の処理（正規化）
    # 入力をDataFrame形式に変換
    rating_df = pd.DataFrame([rating_query], columns=['平均評価'])
    # 学習済みSCALERモデルで変換
    user_rating_vector = scale_rating(rating_df, column_name='平均評価', fit_mode=False)

    # 3. 結合
    # ユーザーベクトルを横方向に連結
    user_final_vector = np.concatenate([user_category_vector, user_rating_vector], axis=1)
    
    return user_final_vector.flatten() # 1次元配列に変換して返す



def create_user_vector_pattern2(category_query, review_query):
    """
    パターン2: カテゴリ + レビュー のユーザーベクトルを生成する
    """
    # 1. カテゴリの処理（ワンホットエンコーディング）
    # 入力をPandas DataFrame形式に変換
    category_df = pd.DataFrame([category_query], columns=['ジャンル'])
    # 学習済みOHEモデルで変換
    user_category_vector = encode_category(category_df, column_name='ジャンル', fit_mode=False)

    # 2. レビューの処理（TF-IDFベクトル化）
    # 入力をDataFrame形式に変換
    review_df = pd.DataFrame([review_query], columns=['レビュー本文'])
    # 学習済みTFIDFモデルで変換
    user_review_vector = vectorize_review(review_df, column_name='レビュー本文', fit_mode=False)

    # 3. 結合
    # ユーザーベクトルを横方向に連結
    user_final_vector = np.concatenate([user_category_vector, user_review_vector], axis=1)
    
    return user_final_vector.flatten() # 1次元配列に変換して返す



def create_user_vector_pattern3(category_query, rating_query, review_query):
    """
    パターン3: カテゴリ + レビュー + 星の数 のユーザーベクトルを生成する
    """
    # 1. カテゴリの処理（ワンホットエンコーディング）
    # 入力をPandas DataFrame形式に変換
    category_df = pd.DataFrame([category_query], columns=['ジャンル'])
    # 学習済みOHEモデルで変換
    user_category_vector = encode_category(category_df, column_name='ジャンル', fit_mode=False)

    # 2. 星の数の処理（正規化）
    # 入力をDataFrame形式に変換
    rating_df = pd.DataFrame([rating_query], columns=['平均評価'])
    # 学習済みSCALERモデルで変換
    user_rating_vector = scale_rating(rating_df, column_name='平均評価', fit_mode=False)

    # 3. レビューの処理（TF-IDFベクトル化）
    # 入力をDataFrame形式に変換
    review_df = pd.DataFrame([review_query], columns=['レビュー本文'])
    # 学習済みTFIDFモデルで変換
    user_review_vector = vectorize_review(review_df, column_name='レビュー本文', fit_mode=False)

    # 4. 結合
    # ユーザーベクトルを横方向に連結
    user_final_vector = np.concatenate([user_category_vector, user_rating_vector, user_review_vector], axis=1)
    
    return user_final_vector.flatten() # 1次元配列に変換して返す